<a href="https://colab.research.google.com/github/Charritocdt93/ProyectoFinal_IADardos/blob/main/ProyectoFinal_IADardos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalaciones

In [1]:
!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet

### Importaciones

In [2]:
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

### Obtener LLM y Entrenarlo

In [3]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Usar Pipeline de Hugging Face

In [4]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

### Embeddings

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

### Template

In [6]:
template = """
[INST] <>
Act as a Machine Learning engineer who is teaching high school students.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

### Cargar Paginas

In [7]:
urls = [
    "https://www.versus.es/reglas-de-dardos",
    "https://www.tododardos.com/reglas-campeonatos/"
]

loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

len(documents)

2

### Reducir el numero de chuck

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_chunks = text_splitter.split_documents(documents)

len(texts_chunks)

65

### Ingesta de datos

In [9]:
db = Chroma.from_documents(texts_chunks, embeddings, persist_directory="db")

### Platilla de Consulta

In [14]:
template = """
[INST] <>
IADarts te responde:
<>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

### Probar IA

In [15]:
query = "¿Cuantos dardos se usan para jugar?"
result_ = qa_chain(
    query
)
result = result_["result"].strip()


display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>¿Cuantos dardos se usan para jugar?</b>

<p>Los juegos de dardos son un deporte competitivo donde dos o más jugadores compiten entre sí para obtener el mayor puntaje. En este deporte, se utilizan dardos de madera o plástico y se lanzan desde una distancia fija hacia una diana electrónica. La diana tiene varios sectores de diferentes colores y puntuaciones asignadas a cada uno.

El objetivo del juego es llegar a un puntaje de 301 o más puntos antes que el otro jugador o equipo. Para lograr esto, los jugadores pueden lanzar dardos individuales o combinados, dependiendo de las reglas del juego.

En general, se utilizan tres dardos por turno, pero el número de dardos que se utilizan varía según las reglas del juego. Además, los jugadores pueden decidir si quieren lanzar todos sus tres dardos en un solo turno o si preferen lanzarlos individualmente.

La puntuación se calcula sumando los puntos obtenidos por cada dardo que golpea un sector de la diana. Si un dardo no golpea ningún sector, no se puntúa.

En algunas variantes del juego, se pueden utilizar diferentes tipos de dardos, como los dardos de metal o los dardos con punterías. También hay diferentes tipos de dianas, como las dianas de madera o las dianas de plástico.

En conclusion, el número de dardos que se utilizan para jugar varía según las reglas del juego, pero generalmente se utilizan tres dardos por turno.</p>

### Follow-Up Q/A

In [23]:
custom_template = """Usted es Asistente de IADarts. Dada la
siguiente conversación y una pregunta de seguimiento, reformule la pregunta de seguimiento
para que sea una pregunta independiente. Al final de la pregunta independiente añade lo siguiente
Responda a la pregunta en inglés". Si no conoce la respuesta, responda "Lo siento, no tengo suficiente información".
Historial de chat:
{chat_history}
Entrada de seguimiento: {pregunta}
Pregunta independiente:
"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT,
)

query = "¿Que juegos ahi?"

result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

### Mensajes en memoria

In [25]:
memory.chat_memory.messages

[HumanMessage(content='¿Que juegos ahi?'),
 AIMessage(content=' The games mentioned are 501, 701, 901 and 1001. They have the same rules as 301, except that each player starts with 501 points or 701 points, etc. To illustrate the rules, we will use 301.')]

### Gradio Chat UI

In [26]:
def querying(query, history):
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=db.as_retriever(search_kwargs={"k": 2}),
      memory=memory,
      condense_question_prompt=CUSTOM_QUESTION_PROMPT,
  )

  result = qa_chain({"question": query})
  return result["answer"].strip()

In [27]:
iface = gr.ChatInterface(
    fn = querying,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="What is GenAI Ecosystem?", container=False, scale=7),
    title="HiberusBot",
    theme="soft",
    examples=["Why Hiberus has created GenAI Ecosystem?",
              "What is GenAI Ecosystem?"],

    cache_examples=True,
    retry_btn="Repetir",
    undo_btn="Deshacer",
    clear_btn="Borrar",
    submit_btn="Enviar"

    )

iface.launch(share=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching examples at: '/content/gradio_cached_examples/15'
Caching example 1/2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching example 2/2
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9a1ef99f0f1c0935cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
